In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/awxlong/Desktop/my-studies/hpc_exps/')


In [ ]:
from HistoMIL.MODEL.Image.MIL.CAMIL.model import CAMIL
from HistoMIL.MODEL.Image.MIL.CAMIL.paras import CAMILParas

In [ ]:
import h5py
from sklearn.metrics import pairwise_distances
import numpy as np
from scipy.spatial import distance
import torch
import os
from sklearn import preprocessing
import pickle
import pandas as pd
from HistoMIL import logger
import logging
logger.setLevel(logging.INFO)
import pdb

In [ ]:
# wsi_coords = h5py.File('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Patch/224_224/tcga_folder_4.TCGA-A8-A086-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2623.svs.h5', 'r')
# wsi_coords = wsi_coords['coords']
# wsi_coords2 = h5py.File('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Patch/224_224/tcga_folder_4.TCGA-AA-A01X-01Z-00-DX1.7433F54C-2A79-467A-8FEA-638AE48F42A0.svs.h5')

In [ ]:
# wsi_feats = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet18/tcga_folder_3.TCGA-A8-A086-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2623.svs.pt')
# wsi_feats2 = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet18/tcga_folder_4.TCGA-AA-A01X-01Z-00-DX1.7433F54C-2A79-467A-8FEA-638AE48F42A0.svs.pt')

In [ ]:
# features = []
# features.append(wsi_feats.cpu().numpy())
# features.append(wsi_feats2.cpu().numpy())

In [ ]:
# np.vstack(features).shape
# wsi_coords.shape
# wsi_feats.shape

In [ ]:
#--------------------------> init machine and person

machine_cohort_loc = '/Users/awxlong/Desktop/my-studies/hpc_exps/User/CRC_machine_config.pkl'#  f"{args.cohort_dir}/User/{args.localcohort_name}_machine_config.pkl"
with open(machine_cohort_loc, "rb") as f:   # Unpickling
    [data_locs,exp_locs,machine,user] = pickle.load(f)


    

In [ ]:
def save_hdf5(output_path, asset_dict, attr_dict=None, mode='a'):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1,) + data_shape[1:]
            maxshape = (None,) + data_shape[1:]
            dset = file.create_dataset(key, shape=data_shape, maxshape=maxshape, chunks=chunk_shape, dtype=data_type)
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0]:] = val
    file.close()
    return output_path

def compute_adj_coords(wsi_coords, wsi_feats, wsi_name, adj_coord_save_path, adj_matrix_save_path, force_recalc = False):
        # output_path_file = os.path.join(save_path + wsi_name + '.h5')
        # output_path_file = data_locs.abs_loc('feature') + f'{encoder}_adj_dictionary/{wsi_name}.h5'
        if not os.path.exists(f'{adj_matrix_save_path}{wsi_name}.pt') or force_recalc: 
             
            patch_distances = pairwise_distances(wsi_coords, metric='euclidean', n_jobs=1)
            neighbor_indices = np.argsort(patch_distances, axis=1)[:, :16]
            rows = np.asarray([[enum] * len(item) for enum, item in enumerate(neighbor_indices)]).ravel()
            columns = neighbor_indices.ravel()
            values = []
            coords = []
            for row, column in zip(rows, columns):
                    m1 = np.expand_dims(wsi_feats[int(row)], axis=0)
                    # pdb.set_trace()
                    m2 = np.expand_dims(wsi_feats[int(column)], axis=0)
                    value = distance.cdist(m1.reshape(1, -1), m2.reshape(1, -1), 'cosine')[0][0]
                    values.append(value)
                    coords.append((row, column))
            
            # mode = 'a'
            values = np.reshape(values, (wsi_coords.shape[0], neighbor_indices.shape[1]))
            
            coords = np.array(coords)
            
            asset_dict = {'adj_coords': coords, 'similarities': values, 'indices': neighbor_indices}

            save_hdf5(adj_coord_save_path, asset_dict, attr_dict=None)

            ### compute adjacency matrix
            values = np.nan_to_num(values)

            Idx = neighbor_indices[:, :8]
            rows = np.asarray([[enum] * len(item) for enum, item in enumerate(Idx)]).ravel()

            columns = Idx.ravel()

            neighbor_matrix = values[:, 1:]

            normalized_matrix = preprocessing.normalize(neighbor_matrix, norm="l2")

            similarities = np.exp(-normalized_matrix)

            values = np.concatenate((np.max(similarities, axis=1).reshape(-1, 1), similarities), axis=1)

            values = values[:, :8]

            values = values.ravel().tolist()

            sparse_coords= list(zip(rows, columns))

            # sparse_matrix = torch.sparse_coo_tensor(sparse_coords, values, (wsi_feats2.shape[0], wsi_feats2.shape[0]))

            indices = torch.tensor(sparse_coords, dtype=torch.long).t()
            # values = torch.tensor(values, dtype=torch.float32)
            values = torch.FloatTensor(values)
            sparse_matrix = torch.sparse.FloatTensor(indices, values, torch.Size([wsi_feats.shape[0], wsi_feats.shape[0]]))

            torch.save(sparse_matrix, f'{adj_matrix_save_path}{wsi_name}.pt')
            logger.info(f'Adjacency matrix stored at {adj_matrix_save_path}')
        else:
             logger.info(f'Adjacency matrix already exists at: {adj_matrix_save_path}{wsi_name}.pt')
        # return np.array(coords), values, neighbor_indices, sparse_matrix


In [ ]:
cohort_name = 'CRC'
idx = 1
local_cohort_dir = f'{exp_locs.root}Data/local_cohort_{cohort_name}_{idx}.csv'
df = pd.read_csv(local_cohort_dir)

step_size = 224
encoder = 'resnet18'

wsi_coord_root = data_locs.abs_loc('patch') + f'{step_size}_{step_size}/'
wsi_feats_root = data_locs.abs_loc('feature') + f'{encoder}/'

h5_path_root = data_locs.abs_loc('feature') + f'{encoder}_adj_dictionary/'
sparse_matrix_root = data_locs.abs_loc('feature') + f'{encoder}_adj_matrix/'

os.makedirs(h5_path_root, exist_ok=True)
os.makedirs(sparse_matrix_root, exist_ok=True)

for i in range(df.shape[0]):
    wsi_name = f'{df.loc[i, "folder"]}.{df.loc[i, "filename"]}'
    wsi_coords_name = f'{wsi_name}.h5'
    wsi_feats = f'{wsi_name}.pt'

    wsi_coords_dir = f'{wsi_coord_root}{wsi_coords_name}'
    wsi_feats_dir = f'{wsi_feats_root}{wsi_feats}'
    if os.path.exists(wsi_coords_dir) and os.path.exists(wsi_feats_dir):
            # Load the slide and its coordinates
            wsi_coordinates = h5py.File(wsi_coords_dir)
            wsi_coordinates = wsi_coordinates['coords']
            wsi_features = torch.load(wsi_feats_dir)  
            
            adj_coords_save_path = f'{h5_path_root}{wsi_name}.h5' # data_locs.abs_loc('feature') + f'{encoder}_adj_dictionary/{wsi_name}.h5'
            adj_matrix_save_path = f'{sparse_matrix_root}{wsi_name}.pt'
            
            # Process the slide and its coordinates
            compute_adj_coords(wsi_coords = wsi_coordinates, 
                               wsi_feats = wsi_features,
                               wsi_name = wsi_name,
                               adj_coord_save_path=adj_matrix_save_path,
                               adj_matrix_save_path=adj_coords_save_path,
                               force_recalc=False
                               )
            
             
    else:
            # Do nothing and continue to the next iteration
        continue

In [ ]:


adj_coords, similarities, neighbor_indices, sparse_matrix = compute_adj_coords(wsi_coords=wsi_coords2['coords'], wsi_feats=wsi_feats2)

        

In [ ]:
print(adj_coords.shape, similarities.shape, neighbor_indices.shape) # mismatched coords with feats # (248288, 2) (15518, 16) (15518, 16) 

In [ ]:
print(adj_coords.shape, similarities.shape, neighbor_indices.shape) # matched coords with feats # (248288, 2) (15518, 16) (15518, 16) 

In [ ]:
print(adj_coords.shape, similarities.shape, neighbor_indices.shape) # mismatched coords with feats # (248288, 2) (15518, 16) (15518, 16) 

In [ ]:
neighbor_indices = neighbor_indices

values = similarities


In [ ]:
sparse_matrix

In [ ]:
torch.save(sparse_matrix, f'{sparse_matrix_save_path}temp_sparse_matrix.pt')

In [ ]:
tst = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni_adj_matrix/temp_sparse_matrix.pt')
tst

In [ ]:
default_paras = CAMILParas()
default_paras.input_shape = 512
rand_tensor = torch.rand(1, 1, 512)# .to('mps')
model = CAMIL(paras=default_paras)
# model.to('mps')



In [ ]:
out, alpha, k_alpha = model([wsi_feats2, sparse_matrix])